# Segmentando clientes

En esta notebook se verá una forma sobre como segmentar a los clientes, cuando cuenta con una gran cantidad de estos sobre una base de datos de una gran (?) cantidad de dimensiones.

Empezaremos cargando el entorno e instalando los requerimentos necesarios.

In [ ]:
#%pip install scikit-learn==1.3.2
#%pip install seaborn==0.13.1
#%pip install numpy==1.26.4
#%pip install matplotlib==3.7.1
#%pip install umap
#%pip install umap-learn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from umap import UMAP
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.ensemble import  RandomForestClassifier
from sklearn.impute import SimpleImputer

import pickle

In [2]:
base_path = 'C:/Users/c678456/Desktop/Ian/Maestría/Especializacion/2do_cuatrimestre/DMEyF/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_fe.csv'

In [3]:
# Datos.
df_train = pd.read_csv(dataset_path + dataset_file)

In [9]:
# Modelo.
filename = modelos_path + 'exp_206_rf_100_imputacion_media.sav'
model_rf = pickle.load(open(filename, 'rb'))

Solo segmentaremos a los clientes de abril, dado que necesitaremos variables en algún momento agregar variables históricas para entender su comportamiento previo a la **baja**

In [4]:
# Filtro los cuatro meses de entrenamiento.
mes_train = [202101,202102,202103,202104]
df_train = df_train[df_train['foto_mes'].isin(mes_train)]

A su vez, nos concentraremos en el fenómeno de la **baja**. No importa si es en un mes o si es en dos.

In [5]:
Xtrain = df_train
ytrain = Xtrain["clase_ternaria"].map(lambda x: 0 if x == "CONTINUA" else 1)
Xtrain = Xtrain.drop("clase_ternaria", axis=1)

del(df_train)

Lo primero que necesitamos es saber cuando un cliente es parecido a otro. Todos los clientes que son parecidos los juntaremos en un **segmento** y sobre esos segmentos haremos perfilados o profiles para entender que los caracteriza.

Saber cuando un cliente es parecido a otro no es algo tan simple en un problema de alta dimensionalidad, lo que se suele llamar **curse of dimensionality**. Las cosas no son lo mismo más allá de nuestras 3 escasas dimensiones. Para muestra, un botón: https://www.youtube.com/watch?v=mceaM2_zQd8

Por esto, es conveniente usar herramientas que nos ayuden a no tener que hacer distancias euclídeas.

Empecemos con una ayuda con nos dan nuestros amigos, los **rf** con una original matriz de distancias (https://en.wikipedia.org/wiki/Distance_matrix)

La **Random Forest Distance** es una matriz derivada del algoritmo Random Forest,

+ Se utiliza para medir la similitud entre pares de puntos de datos. Esta métrica se basa en la idea de cuántas veces dos puntos de datos terminan en la misma hoja de un árbol en un **rf**, entre los múltiples árboles que se generan.

+ Cómo se Calcula?

 1. Se entrena un **rf**.
 2. Para cada par de clientes $c_i$ y $c_j$, se observa cuántas veces caen en la misma hoja a través de todos los árboles del bosque. Luego se agrega en la posición $i$, $j$ de una matriz el número de veces que coincidieron esos dos clientes en un nodo terminal.
 3. Se calcula la matriz de distancia como **1 - proporción de veces que caen en la misma hoja**. Ejemplo: Si caen en la misma hoja el 90% de las veces, su distancia será 0.1. Si nunca caen en la misma hoja, la distancia es 1.

Veamoslo aplicado en nuestro caso

Para trabajar con este algoritmo no vamos a trabajar con todos los datos. Usted decida con cuantos trabajar. En este caso, tomaremos todos los **baja** y unos 2000 **continua**, ya que queremos como interactuan los segmentos de clientes que se van con los que se quedan.


In [7]:
np.random.seed(17)
continua_sample = ytrain[ytrain == 0].sample(2000).index
bajas_1_2 = ytrain[ytrain == 1].index
rf_index = continua_sample.union(bajas_1_2)

Xtrain_rf = Xtrain.loc[rf_index]
ytrain_rf = ytrain.loc[rf_index]

Entrenamos un simple **rf**, despliegue sus herramientas aprendidas para contar con un buen modelo


In [ ]:
#imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
#Xtrain_rf_imp = imp_mean.fit_transform(Xtrain_rf)

#model = RandomForestClassifier(n_estimators=100, max_features=20, min_samples_leaf=400, random_state=17 )
#model.fit(Xtrain_rf_imp, ytrain_rf)

model_rf.fit(Xtrain_rf,ytrain_rf)

Armamos (copiamos de internet) una función que nos calcule la matriz de distancias

In [11]:
def distanceMatrix(model, X):

    terminals = model.apply(X)
    nTrees = terminals.shape[1]

    a = terminals[:,0]
    proxMat = 1*np.equal.outer(a, a)

    for i in range(1, nTrees):
        a = terminals[:,i]
        proxMat += 1*np.equal.outer(a, a)

    proxMat = proxMat / nTrees

    return proxMat.max() - proxMat

#md = distanceMatrix(model, Xtrain_rf_imp)
md = distanceMatrix(model_rf, Xtrain_rf)

Veamos como se ve. Recuerde que significa que un número esté cerca de 0 o que esté cerca de 1.

Para poder visualizar la matriz, utilizaremos un embedding. Los *embeddings* son representaciones vectoriales de datos en un espacio de menor dimensión

Podría utilizar un **PCA** con ese fin, pero en los últimos años hay mejores algoritmos como **t-sne** o **umap**.

UMAP (Uniform Manifold Approximation and Projection) es una técnica avanzada para la reducción de dimensionalidad y visualización de datos en espacios de menor dimensión, que busca mantener al máximo la estructura de los datos en alta dimensión.

1. **Preservación de la Estructura Global y Local**:
   - Intenta preservar tanto la estructura local (relaciones cercanas) como la global (estructura general) de los datos al proyectarlos en un espacio de menor dimensión.

2. **Basado en Manifold Learning**:
   - UMAP asume que los datos de alta dimensión se encuentran en un espacio de menor dimensión (un "manifold") y busca proyectar esos datos de manera que se mantenga esa estructura subyacente.

3. **Velocidad y Escalabilidad**:
   - UMAP es más rápido y escalable en comparación con técnicas similares como t-SNE, especialmente en conjuntos de datos grandes.

4. **Control sobre la Estructura**:
   - UMAP permite al usuario ajustar parámetros que controlan la preservación de la estructura local y global, como el número de vecinos cercanos (n_neighbors) y la distancia mínima entre puntos (min_dist).

Una característica adicional, es que cuenta con la posibilidad de recibir de entrada una matriz de distancia.

Veamos los datos por primera vez:

In [ ]:
embedding_rf = UMAP(
  n_components=2,
  n_neighbors=50,
  metric="precomputed",
  random_state=17,
).fit_transform(md)

plt.scatter(embedding_rf[:,0], embedding_rf[:,1])

Agreguemos la dimensión de la probabilidad de salida, para ver donde cree el modelo que se encuentran los **bajas**.

In [ ]:
#class_index = np.where(model.classes_ == 1)[0]
#prob_baja = model.predict_proba(Xtrain_rf_imp)[:,class_index]

class_index = np.where(model_rf.classes_ == 1)[0]
prob_baja = model_rf.predict_proba(Xtrain_rf)[:,class_index]

plt.scatter(embedding_rf[:,0], embedding_rf[:,1], c=prob_baja)
plt.colorbar()
plt.show()

Vamos a partir de este embedding para segmentar a los clientes. Puede utilizar cualquier técnica, los datos ya son simples para cualquier algoritmo. Utilizaremos uno bastante estandar

In [ ]:
hdb = DBSCAN(eps=0.3)
y = hdb.fit(embedding_rf)

# Agregar etiquetas de clúster
unique_labels = set(y.labels_)
for label in unique_labels:
    # Filtrar los puntos de cada clúster
    cluster_points = embedding_rf[y.labels_ == label]
    
    # Calcular el centroide del clúster para colocar la etiqueta
    centroid = cluster_points.mean(axis=0)
    
    # Colocar el número del clúster en el gráfico
    plt.text(centroid[0], centroid[1], str(label), fontsize=12, fontweight='bold', color='black')
    
plt.scatter(embedding_rf[:, 0], embedding_rf[:, 1], c=y.labels_, cmap='tab10')
plt.tight_layout()
plt.show()

In [ ]:
# Aplana el array de prob_baja para que sea unidimensional
prob_baja_flat = prob_baja.ravel()  # O usa prob_baja.flatten() si prefieres

# Verifica que ahora ambos arrays sean unidimensionales y tengan la misma longitud
print(f"Dimensiones de etiquetas de clusters: {y.labels_.shape}")
print(f"Dimensiones de probabilidad de baja (aplanado): {prob_baja_flat.shape}")

# Crear el DataFrame si las dimensiones coinciden
if y.labels_.shape[0] == prob_baja_flat.shape[0]:
    df_clusters = pd.DataFrame({
        'cluster': y.labels_,  # Etiquetas de los clusters del DBSCAN
        'prob_baja': prob_baja_flat  # Probabilidades de baja de cada cliente (aplanado)
    })

    # Paso 2: Calcular el promedio de probabilidad de baja por cluster
    cluster_avg_prob_baja = df_clusters.groupby('cluster')['prob_baja'].mean()

    # Mostrar los resultados
    print(cluster_avg_prob_baja)
else:
    print("Las dimensiones de 'y.labels_' y 'prob_baja' aún no coinciden.")

Veamos cuantos cluster detecto y cuantos clientes tiene cada uno

Los que tienen -1, son considerados outliers.

Por último necesitamos alguna forma de saber que hace a cada cluster distinto del otro. Para esto utilizaremos modelos (**rf**) que buscan separar los cluster uno a uno del resto de los datos.

Por cada modelo, miraremos cuales son las variables más importantes que separan los datos para luego caracterizarlos:

Manos a la obra:

In [18]:
df_embedding = pd.DataFrame(embedding_rf, columns=['embedding_1', 'embedding_2'])
df_embedding['cluster'] = y.labels_

clusters = df_embedding['cluster'].unique()

important_features_by_cluster = {}

for cluster in clusters:
  y_binary = (df_embedding['cluster'] == cluster).astype(int)

  model = RandomForestClassifier(random_state=17)
  #model.fit(Xtrain_rf_imp, y_binary)
  model.fit(Xtrain_rf, y_binary)

  importances = model.feature_importances_
  feature_names = Xtrain_rf.columns

  indices = np.argsort(importances)[::-1]
  important_features_by_cluster[cluster] = [feature_names[i] for i in indices]

Y exploramos cuales son las variables importantes por cada cluster

In [ ]:
for cluster, features in important_features_by_cluster.items():
  print(f"Cluster {cluster} vs. Resto:")
  for feature in features[:15]:
    print(f"  - {feature}")

Luego resta, analizar los estadísticos de los datos de cada cluster para las variables importantes, comparar sus distribuciones con histogramas, boxplots, pivot tables, etc.


Junto con el diccionaro de datos, de sentido a lo que ve a través del análisis multivariado de datos.

Obviamente, esto es apenas mas que una semilla. Agregue las variables históricas, juegue con los **rf** y genere una segmentación pro, que encante a Miranda


#### 1. Creo el dataframe a analizar.

In [107]:
df_analizar = Xtrain_rf.copy()
df_analizar['cluster'] = y.labels_

#### 2. Análisis de cada clúster.

In [110]:
descriptivo_cluster = {}
for cluster, features in important_features_by_cluster.items():
    #print("\n\nCLUSTER {}\n\n".format(cluster))
    features_filtrar = features[:15]
    df = df_analizar[df_analizar["cluster"] == cluster][features_filtrar]
    descriptivo_cluster[cluster] = df.describe()

##### a. Clúster 0 (vs clúster 4).

In [ ]:
descriptivo_cluster[0]

In [ ]:
# No son trabajadores (payroll bajo) por ende, no tienen apego por el banco.
# Tienen pasivos con el banco.
# No suelen tener mucha plata en la caja de ahorro (hasta el 75% tiene $25,583.28).
# Suelen deber por la TdeC (media -$28,864.39 ante un pago mínimo de $6,676.71)

In [ ]:
descriptivo_cluster[4]

In [ ]:
# Si comparamos el clúster 0 con el 4 (CONTINUA CLARAMENTE), vemos que...
# - Son claramente Trabajadores con aumentos de sueldo.
# - Hacen transacciones bancarias.
# - Usan la tarjeta de crédito de gran manera.
# - Hacen extracciones del autoservicio, etc.

##### b. Clúster 1 vs clúster 3 (son parecidos).

In [ ]:
descriptivo_cluster[1]

In [ ]:
# Casi sin uso de la caja de ahorro (de hecho, va descendiendo).
# Casi sin uso de la tarjeta de crédito VISA (de hecho, casi sin saldo, sin consumos, casi todo el limite descubierto).
# No genera pasivos (no usa los productos financieros).
# Casi sin movimientos.
# La antiguedad es menor que el clúster 0 y 4.

In [ ]:
descriptivo_cluster[3]

In [ ]:
# Parece no tener pasivos (falta de productos financieros).
# No usa la caja de ahorro ni las tarjetas de crédito.
# Casi sin saldo en la cuenta.
# Casi no hace transacciones bancarias.



# ------------> Muy parecido al clúster 1... BUSCAR LA DIFERENCIA... en otras variables? hacer otro árbol?
# --------------> RECOMENDACION: PERDERLOS.

In [ ]:
# Paso 1: Obtener las 10 variables más importantes para los clusters 1 y 3
top_10_cluster_1 = [feature for feature in important_features_by_cluster[1][:10]]
top_10_cluster_3 = [feature for feature in important_features_by_cluster[3][:10]]

# Paso 2: Unir las variables de los dos clusters y eliminar duplicados
unique_top_features = list(set(top_10_cluster_1 + top_10_cluster_3))

# Paso 3: Filtrar los datos para los clusters 1 y 3
df_filtered = df_analizar[df_analizar['cluster'].isin([1, 3])]

# Paso 4: Hacer un boxplot para cada una de las variables más importantes sin mostrar outliers
for feature in unique_top_features:
    plt.figure(figsize=(8, 6))
    df_filtered.boxplot(column=feature, by='cluster', grid=False)#, showfliers=False)
    plt.title(f'Boxplot of {feature} by Cluster 1 and 3 (No Outliers)')
    plt.suptitle('')  # Remover el título que agrega pandas por defecto
    plt.xlabel('Cluster')
    plt.ylabel(f'{feature} values')
    plt.show()

##### c. Clúster 5.

In [ ]:
descriptivo_cluster[5]

In [ ]:
# Uso de prestamos personales (con pendiente positiva).
# Margen por Activos positivo.
# No suele tener mucha plata en la caja de ahorros.
# No son trabajadores.
# Posible competencia vs otros bancos ante tasas más favorables?

##### d. Clúster 6.

In [ ]:
descriptivo_cluster[6]

In [ ]:
# Perfiles muy importantes (resaltarlos) ya que son muy rentables para el banco y manejan grandes volúmenes de activos.
# De hecho, dicha rentabilidad, está en aumento (gran consumo de productos financieros).
# Lo mismos con el margen por activos (cobro intereses).
# Ahora bien, tienen saldo en cuenta negativo, cuenta corriendo negativo,  y los pasivos_margen están cayendo.
# La presencia de saldos negativos y la variabilidad en mcuentas_saldo y mcuenta_corriente sugiere que los clientes en el Cluster 6 realizan transacciones de gran volumen, lo que podría ser diferente a la frecuencia alta pero de menor magnitud observada en Cluster 5.

##### e. Clúster 7.

In [ ]:
descriptivo_cluster[7]

In [ ]:
# Tienen payroll, pero bajo.
# Se les cobra muchas comisiones de mantenimiento.
# No tienen mucha plata en la caja de ahorro.
# Realizan algunas transacciones.
# Son más bien gente de alta edad y con mucha antigüedad.

# -------------> Comparación con el clúster 4.
# Los clientes del Cluster 4 tienen ingresos más altos y, por ende, mayor variedad de productos financieros con mejores condiciones?????. 
# A estos se les puede ofrecer cuentas premium o productos de inversión personalizados ??????

##### f. Clúster 8 vs. Clúster 9 (son parecidos).

In [ ]:
descriptivo_cluster[8]

In [ ]:
# Son clientes con TC de hace muy poco tiempo (Usan ambas tarjetas, tanto Visa como Mastercard nuevos) ----> Analizar deudas por la TC.
# Con saldo total en todas las cuentas negativo (especialmente, en la cuenta corriente) ---> Analizar préstamos, sobregiros. Se puede deber a problemas financieros o simplemente de un uso intensivo de las cuentas corrientes.
# Tienen mactivos_margen (intereses), lo cual habla de cobro de intereses.
# Les cobran "comisiones otras" (por sobregiros? cheques rechazados? descubierto? pagos automáticos? uso de tarjetas?).

# Estos intereses/comisiones se puede deber para el mantenimiento de sus servicios, debido que son clientes nuevos. (servicios ---> Tarjeta de Crédito, Uso de descubierto).
# Porque no usan la tarjeta?

# -----------> Solución: Bajar los intereses/comisiones por saldos negativos? O son clientes que se fueron haciendo morosos?

In [ ]:
descriptivo_cluster[9]

In [ ]:
# También son clientes nuevos (como el clúster 8).
# Tienen tarjeta VISA y MASTERCARD hace poco tiempo.
# Generan una rentabilidad negativa al banco! (cluster 8 aunque sea genera comisiones) pero rentabilidad anual.
# Suelen tener al menos una caja de ahorro y una cuenta corriente.

# ----------------> Ahondar en la diferencia con el clúster 8.

In [ ]:
#Perfil de los clientes en riesgo de abandono
#Clúster 8:
#Perfil: Clientes con una relación moderada con el banco, usando diversos productos financieros (como cuentas y tarjetas), pero con una situación financiera comprometida (saldos negativos y deudas). Están pagando comisiones relativamente altas y pueden estar buscando mejores opciones en otros bancos.
#Clúster 9:
#Perfil: Clientes recientes, que han abierto productos financieros hace poco tiempo, con una actividad y rentabilidad baja. Estos clientes aún no han consolidado una relación fuerte con el banco, y si no ven mejoras o beneficios claros, podrían decidir abandonar rápidamente.

#### 3. Análisis multivariado.

In [84]:
# Mantener solo las 10 primeras variables más importantes por cluster
top_10_features_per_cluster = set()

for cluster, features in important_features_by_cluster.items():
    top_10_features_per_cluster.update(features[:10])

# Convertimos el set en una lista.
top_10_features_list = list(top_10_features_per_cluster)

In [ ]:
# Función para eliminar outliers basada en IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Crear boxplots sin outliers para cada una de las variables seleccionadas
for feature in top_10_features_list:
    df_filtered = remove_outliers(df_analizar, feature)
    plt.figure(figsize=(10, 6))
    sns.boxplot(x="cluster", y=feature, data=df_filtered)
    plt.title(f"Boxplot de {feature} aperturado por Cluster (sin outliers).")
    plt.show()